In [6]:
# extract features from train/valid/test files

In [7]:
from pprint import pprint
from features.cosine_similarity import cosine_similarity
from features.content_features import *
from features.user_features import *
from features.structural_features import *
from features.sentiment_features import *
from data_helper import *

In [8]:
train_file = '../data/Experiment/Train1.csv'
valid_file = '../data/Experiment/Valid1.csv'
test_file = '../data/Experiment/Test1.csv'

In [9]:
idf_file = '../data/idf.csv'

train_feat_file = '../data/Experiment/train_features.csv'
valid_feat_file = '../data/Experiment/valid_features.csv'
test_feat_file = '../data/Experiment/test_features.csv'

pos_file = '../data/positive-words.txt'
neg_file = '../data/negative-words.txt'
term_to_idf_dict = init_tf_idf_dict(idf_file)
pos_dict, neg_dict = load_sentiment_lexicon(pos_file, neg_file)
labelTable={1:'OQ',2:'RQ',3:'CQ',4:'FD',5:'FQ',\
            6:'IR',7:'PA',8:'PF',9:'NF',10:'GG',11:'JK',12:'O'}
labelTable[5]

'FQ'

In [10]:
for in_file, out_file in zip([train_file, valid_file, test_file], [train_feat_file, valid_feat_file, test_feat_file]):
    with open(in_file, encoding='cp437') as fin, open(out_file, 'w') as fout:
        utterances = []
        labels = []
        uoas = []
        # skip the first line
        next(fin)
        for line in fin:
            if line != '\n':
                tokens = line.strip().split(',')
#                 print(tokens)
                label_index = tokens[0:13].index("1")
                utterances.append(tokens[14])
                uoas.append(tokens[19])
            else:
                # extract features
                
                # content based features
                _, init_sim, thread_sim = cosine_similarity("", utterances, term_to_idf_dict)
                qm = question_mark(utterances)
                dup = duplicate(utterances)
                wh = W5H1(utterances)

                # structural features
                abs_pos = [idx + 1 for idx in range(len(utterances))]
                norm_pos = [pos / len(utterances) for pos in abs_pos]
                length, unique_length, unique_stemmed_length = post_length(utterances)

                # user features
#                 ua = user_auth(affiliations)
                is_starter = [1 if uoa == 'User' else 0 for uoa in uoas]

                # sentiment based features
                thx = thank(utterances)
                exclam_mark = exclamation_mark(utterances)
                vf = ve_feedback(utterances)
                ss = sentiment_scores(utterances)
                lexicon_counts = lexicon(utterances, pos_dict, neg_dict)
                
                # write to file
                for i, utterance in enumerate(utterances):
#                     try:
                    fout.write('{}\t{:.4f} {:.4f} {} {} {} {} {:.4f} {} {} {} {} {} {} {} {} {}\n'.format(
                        labels[i],
                        init_sim[i],
                        thread_sim[i],
                        qm[i],
                        dup[i],
                        ' '.join(wh[i]),
                        abs_pos[i],
                        norm_pos[i],
                        length[i],
                        unique_length[i],
                        unique_stemmed_length[i],
                        is_starter[i],
                        thx[i],
                        exclam_mark[i],
                        vf[i],
                        ' '.join(ss[i]),
                        ' '.join(lexicon_counts[i]),
                    ))
#                     except:
#                         print(utterance)
                        
                fout.write('\n')
                utterances = []
                labels = []
                uoas = []